In [1]:
from pysmt.shortcuts import Symbol, And, Equals, Int, Or, Implies, Solver, Not
from pysmt.typing import INT
import itertools

In [2]:
N = [
    [10] + [6]*3 + [9]*6,
    [10]*2 + [6]*3 + [9] + [3]*2 + [9]*2,
    [10]*2 + [3]*2 + [2]*2 + [6, 3] + [7]*2,
    [10]*2 + [4, 3, 3] + [6]*3 + [7]*2,
    [10] + [4]*2 + [3]*2 + [2, 1] + [6]*2 + [7],
    [10, 1, 4, 5, 1, 2] + [3]*2 + [7]*2,
    [10, 10, 2] + [5]*3 + [2] + [3]*3,
    [10]*2 + [2, 5, 5, 1, 9] + [4]*2 + [9],
    [10]*3 + [5]*2 + [9]*2 + [4]*2 + [9],
    [10]*4 + [5]*2 + [9]*4
]

In [3]:
R = range(10)

f = True
s = [[Symbol(f"{i},{j}", INT) for j in R] for i in R]
for i, j in itertools.product(R, R):
        f &= (1 <= s[i][j]) & (s[i][j] <= N[i][j])
f

(((((... & ...) & (... & ...)) & ((... <= ...) & (... <= ...))) & ((1 <= '9,8') & ('9,8' <= 9))) & ((1 <= '9,9') & ('9,9' <= 9)))

In [4]:
def eq(i, j, k):
    return Equals(s[i][j], Int(k))
def neq(i, j, k):
    return Not(eq(i, j, k)) 

for i, j in itertools.product(range(10), range(10)):
    for k in range(1, N[i][j] + 1):
        ok, nok = [], []
        for i_, j_ in itertools.product(range(10), range(10)):
            if abs(i - i_) + abs(j - j_) == k:
                ok.append(eq(i_, j_, k))
            if abs(i - i_) + abs(j - j_) < k and (i, j) != (i_, j_):
                nok.append(neq(i_, j_, k))
        f &= Implies(eq(i, j, k), And(Or(ok), And(nok)))

In [5]:
for (i, j, k) in [(0, 1, 3), (0, 5, 7), (1, 3, 4), (2, 8, 2), (3, 3, 1), (4, 0, 6), (4, 2, 1), (5, 7, 3), (5, 9, 6), (6, 6, 2), (7, 1, 2), (8, 6, 6), (9, 4, 5), (9, 8, 2)]:
    f &= eq(i, j, k)

In [9]:
from functools import reduce
from operator import mul

with Solver() as solver:
    solver.add_assertion(f)
    solver.solve()
    V = [[solver.get_value(s[i][j]).constant_value() for j in R] for i in R]
    for i in range(10):
        for j in range(10):
            print(V[i][j], end="  ")
        print()


4  3  6  2  5  7  1  4  1  5  
1  1  1  4  1  1  1  2  1  2  
7  1  3  2  1  2  6  1  2  1  
2  1  1  1  1  3  4  1  3  1  
6  1  1  2  1  1  1  1  5  1  
2  1  3  1  1  2  1  3  1  6  
1  1  2  1  1  1  2  1  1  1  
1  2  1  1  1  1  1  1  2  1  
5  1  3  1  1  1  6  1  1  5  
1  1  1  2  5  2  3  1  2  3  


In [7]:
sum(reduce(mul, V[i]) for i in R)

102990